In [57]:
import bloqade.stim
import bloqade.tsim

import warnings

import cirq
import numpy as np
import matplotlib.pyplot as plt
import bloqade.cirq_utils as utils
from cirq.contrib.svg import SVGCircuit

from bloqade import squin, cirq_utils

warnings.filterwarnings("ignore")

@squin.kernel
def main():
    q_data = squin.qalloc(7)
    # squin.u3(theta, phi, 0, q_data[6])

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_data[i])

    # apply cz to pairs
    squin.cz(q_data[1], q_data[2])
    squin.cz(q_data[3], q_data[4])
    squin.cz(q_data[5], q_data[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_data[6])

    # apply cz to more pairs
    squin.cz(q_data[0], q_data[3])
    squin.cz(q_data[2], q_data[5])
    squin.cz(q_data[4], q_data[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_data[i])

    # apply MORE cz gates
    squin.cz(q_data[0], q_data[1])
    squin.cz(q_data[2], q_data[3])
    squin.cz(q_data[4], q_data[5])

    squin.sqrt_y(q_data[1])
    squin.sqrt_y(q_data[2])
    squin.sqrt_y(q_data[4])

    """
    Encode auxillatry qbit
    """
    q_aux = squin.qalloc(7)
    squin.h(q_aux[6])

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_aux[i])

    # apply cz to pairs
    squin.cz(q_aux[1], q_aux[2])
    squin.cz(q_aux[3], q_aux[4])
    squin.cz(q_aux[5], q_aux[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_aux[6])

    # apply cz to more pairs
    squin.cz(q_aux[0], q_aux[3])
    squin.cz(q_aux[2], q_aux[5])
    squin.cz(q_aux[4], q_aux[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_aux[i])

    # apply MORE cz gates
    squin.cz(q_aux[0], q_aux[1])
    squin.cz(q_aux[2], q_aux[3])
    squin.cz(q_aux[4], q_aux[5])

    squin.sqrt_y(q_aux[1])
    squin.sqrt_y(q_aux[2])
    squin.sqrt_y(q_aux[4])

    """
    Manual bit flip
    X-gate applied to 6th qbit of q_data

    Then couple with CNOT gate to data and auxillary logical qbit 
    """

    for i in range(7):
        squin.cx(q_data[i], q_aux[i])

    # introduce error by bit-flipping 5th physical qbit in 
    squin.broadcast.measure(q_data + q_aux, key='result')

# plotting circuit diagram (plotting is nicer on Tsim)
tsim_circ = bloqade.tsim.Circuit(main)
tsim_circ.diagram(height=400)

# sampling using Stim (Clifford-only sampling is faster on Stim)
shots = 100
stim_circ = bloqade.stim.Circuit(main)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shots)
print(samples)

[[False  True  True ... False  True False]
 [False  True False ... False False False]
 [False False  True ... False False  True]
 ...
 [ True False False ...  True False False]
 [ True False False ...  True False False]
 [ True  True  True ... False  True  True]]


In [58]:
tsim_circ.diagram(height=400)

In [59]:
s = samples[0]
s


array([False,  True,  True,  True,  True,  True, False, False, False,
        True,  True, False,  True, False])

In [47]:
s[0] ^ s[1] ^ s[2] ^ s[3]


np.False_

In [48]:
True ^ True

False

In [60]:
auxilary_results = samples[:, 7:14]
data_results = samples[:, 0:7]  

In [61]:
stabilizer1 = []
stabilizer2 = []
stabilizer3 = []

for shot in range(shots):
    sample = auxilary_results[shot]
    stab1 = sample[0] ^ sample[1] ^ sample[2] ^ sample[3]
    stabilizer1.append(stab1)
    stab2 = sample[2] ^ sample[3] ^ sample[4] ^ sample[6]
    stabilizer2.append(stab2)
    stab3 = sample[1] ^ sample[2] ^ sample[4] ^ sample[5]
    stabilizer3.append(stab3)

In [62]:
stabilizer1

[np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.

In [63]:
stabilizer2

[np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.

In [64]:
stabilizer3

[np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.

In [65]:
def false_indices(stabilizer1, stabilizer2, stabilizer3):
    return [
        i for i, (a, b, c) in enumerate(zip(stabilizer1, stabilizer2, stabilizer3))
        if not a and not b and not c
    ]


In [66]:
print(false_indices(stabilizer1, stabilizer2, stabilizer3))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
